In [59]:
import pandas as pd
import selenium
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import multiprocessing
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from multiprocessing import Pool
import numpy as np
import time

from selenium.common.exceptions import TimeoutException, ElementNotInteractableException, ElementClickInterceptedException

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path("../lib/").absolute()))

In [45]:
profile = FirefoxProfile()
profile.set_preference("javascript.enabled", True)
driver = webdriver.Firefox(profile)
driver.implicitly_wait(10)
driver.get("https://www.openstreetmap.org/")

In [69]:
def run_scraper(coords):
    profile = FirefoxProfile()
    profile.set_preference("javascript.enabled", True)
    driver = webdriver.Firefox(profile)
    driver.implicitly_wait(10)
    driver.get("https://www.openstreetmap.org/")
    
    # close the popups
    for elem in driver.find_elements_by_class_name('close-wrap'):
        try:
            elem.click()
        except (ElementNotInteractableException, ElementClickInterceptedException) as e:
            pass
    
    for interid, lat, lon, zoom in coords:
        driver.get("https://www.openstreetmap.org/#map=%d/%f/%f" % (zoom, lat, lon))
        time.sleep(3)
        driver.save_screenshot("screenshots/%d*%d.png" % (interid, zoom))
    
    driver.quit()
    

In [65]:
def scrape_pool(all_coords, num_processes=2):
    pool = Pool(num_processes)
    manager = multiprocessing.Manager()
    coords = np.array_split(all_coords, num_processes)

    for i in range(num_processes):
        pool.apply_async(run_scraper, args=(coords[i], ))

    pool.close()
    pool.join()

In [17]:
orig_train = pd.read_csv('train.csv')
orig_test = pd.read_csv('test.csv')

In [40]:
relevant_columns = ['IntersectionId', 'Latitude', 'Longitude']
full_df = pd.concat([orig_train[relevant_columns], orig_test[relevant_columns]]) \
            .drop_duplicates(['IntersectionId'])
full_df['zoom'] = 16

In [42]:
full_df

,IntersectionId,Latitude,Longitude,zoom
0,0,33.79166,-84.43003,16
654,1,33.75094,-84.39303,16
1011,2,33.74014,-84.31389,16
1337,4,33.81767,-84.36696,16
2882,5,33.77124,-84.38884,16
3651,6,33.75712,-84.38424,16
3847,9,33.81100,-84.41280,16
4029,10,33.65287,-84.39762,16
4187,11,33.74027,-84.34924,16
4684,13,33.78177,-84.36868,16


In [71]:
full_df[full_df['IntersectionId'] == 802]

,IntersectionId,Latitude,Longitude,zoom
275971,802,42.34023,-71.10589,16


In [70]:
scrape_pool(full_df.drop_duplicates().values, num_processes=4)